In [4]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl

In [29]:
def exploring_dataset():
    from sklearn.datasets import load_iris # Load Iris Data
    import pandas as pd
    import numpy as np
    
    iris = load_iris() # Creating pd DataFrames
    iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
    target_df = pd.DataFrame(data= iris.target, columns= ['species'])
    
    species = []

    for i in range(len(target_df['species'])):
        if target_df['species'][i] == 0:
            species.append("setosa")
        elif target_df['species'][i] == 1:
            species.append('versicolor')
        else:
            species.append('virginica')

    target_df['species'] = species 
    
    iris_df = pd.concat([iris_df, target_df], axis= 1)
    
    print(iris_df)
    print(target_df)
    
    from sklearn.model_selection import train_test_split  
    from sklearn.linear_model import LogisticRegression
    from sklearn.linear_model import LinearRegression
    
    
    # Converting Objects to Numerical dtype
    iris_df.drop('species', axis= 1, inplace= True)
    target_df = pd.DataFrame(columns= ['species'], data= iris.target)
    iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
    X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
    y= iris_df['sepal length (cm)']

    # Splitting the Dataset 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state= 101)

    # Instantiating LinearRegression() Model
    lr = LinearRegression()

    # Training/Fitting the Model
    lr.fit(X_train, y_train)

    # Making Predictions
    lr.predict(X_test)
    pred = lr.predict(X_test)

    from sklearn import metrics
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import mean_squared_error
        
    # Evaluating Model's Performance
    print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
    print('Mean Squared Error:', mean_squared_error(y_test, pred))
    print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred))) 
    
    
    d = {'sepal length (cm)' : [4.6],
        'sepal width (cm)' : [3.4],
        'petal length (cm)' : [1.4],
        'petal width (cm)' : [0.3],
        'species' : 0}
    test_df = pd.DataFrame(data= d)
    test_df
    
    print(test_df)
    
    pred = lr.predict(X_test)
    
    print('Predicted Sepal Length (cm):', pred[0])
    print('Actual Sepal Length (cm):', 4.6)

In [30]:
#exploring_data = comp.create_component_from_func(exploring_dataset,output_component_file='iris/exploring_component.yaml',base_image='tensorflow/tensorflow:1.11.0-py3')
exploring_data = comp.create_component_from_func(exploring_dataset,output_component_file='iris/exploring_component.yaml',base_image='tensorflow/tensorflow:1.12.0-py3')



In [31]:
def convert_dataset(iris:object):
    species = []

    for i in range(len(iris['target'])):
        if iris['target'][i] == 0:
            species.append("setosa")
        elif iris['target'][i] == 1:
            species.append('versicolor')
        else:
            species.append('virginica')

    iris['species'] = species

In [32]:
convert_data = comp.create_component_from_func(convert_dataset,output_component_file='iris/convert_component.yaml',base_image='tensorflow/tensorflow:1.11.0-py3')

In [33]:
@dsl.pipeline(name='Pipeline Iris',description='Analise de dados do dataset IRIS')
def pipeline_iris():
    exploring = exploring_data()
    #convert = convert_data(exploring_data.output)
    

In [34]:
client = kfp.Client(host='http://localhost:8080/pipeline')
FILE = ''
args = {}
##print(client.list_experiments())
client.create_run_from_pipeline_func(pipeline_iris,arguments=args)

RunPipelineResult(run_id=5543bdb9-a9d2-4c05-a424-cc57556619e8)